In [1]:
%pip install pandas numpy python-calamine openpyxl -U --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import re

load = "KnownSites_manualedit.xlsx"
sink = "KnownSites_normalized.csv"

In [3]:
df = pd.read_excel(load, engine='calamine')
unnamed = [col for col in df.columns if 'Unnamed' in col]
df = df.drop(columns=unnamed)
df.head()

,NAME,GEOGRAPHICAL REGION,GEOGRAPHICAL LOCATION,COORDINATES,TYPE OF SITE,ELEVATION,PERIOD,CULTURAL COMPLEX,ABSOLUTE DATING,STONE ARTIFACTS,PALEOLITHIC HUMAN REMAINS,LITERATURE
0,Abri Kontija 002,Istra,Limski kanal,"45° 8'15""N, 13°43'7"" E",RS,46,UP,G,/,YES,NO,"(Janković et al. 2016, 2017a; b, 2022; Peresan..."
1,Abri Šebrn\r\n,Istria,Ćićarija,"45.33771253384577, 14.162687241350081",RS,750,MES,EMes,8650±9360 C-14 bp or 7610±8400 Cal BC,YES,NO,(Miracle et al. 2000; Komšo 2006: 64; Komšo et...
2,Brjgućeva Loza 1 (Loza),Istra,Kastav,"45° 28' 4"" N, 14° 14' 32"" E",C,510,MES,NaN,NaN,YES,NO,(Malez 1979: 242–43 karta 1:52; Paunović et al...
3,Borik,Dalmacija,Zadar (Oštri rat ili Punta Mika)\r\n,"44° 05'N, 15° 15' E",O,5,MES,NaN,NaN,YES,NO,(Malez 1979: 227 karta 1:76; Paunović et al. 2...
4,Brodski Drenovac,Slavonija,brodsko Posavlje (Pleternica),"45° 13'N, 17° 45'E",O,248,"UP, Mez",NaN,NaN,YES,NO,(Malez 1979: 227 karta 1:50; Komšo 2006: 76)


In [4]:
def normalize(string: str):
    if re.match(r"\d+\.\d+\s*[Ne]*,\s*\d+\.\d+\s*[Ee]*", string.strip()):
        temp = string.upper().replace("N", "").replace("E", "").split(",")
        return tuple(map(float, temp))
    
    elif re.match(r"""\d+[°]\s*\d*[']*\s*\d*[.]*\d*["']*\s*[Nn]\s*[,]{1}\s*\d+[°]\s*\d*[']*\s*\d*["']*[.]*\d*\s*[Ee]*""", string.strip()):
        temp = [re.split('[°\'"]', i) for i in string.split(",")]
        
        return tuple(map(convert, temp))
    
    else:
        return (0, 0)
    
def convert(values: list):
    values = [i.strip() for i in values if i.strip() != ""]
    total = 0
    for i, value in enumerate(values):
        if value.lower() in ["w", "s"]:
            total *= -1
            break
        elif value.lower() in ["e", "n"]:
            break
        if i == 0:
            total += float(value.strip())
        elif i == 1:
            total += float(value)/60
        elif i == 2:
            total += float(value)/3600
    
    return total
        

test = df.copy()
normalized = test['COORDINATES'].apply(normalize)
test.loc[:,'coord_ns'], test.loc[:,'coord_ew'] = list(zip(*normalized))[0], list(zip(*normalized))[1]
test

,NAME,GEOGRAPHICAL REGION,GEOGRAPHICAL LOCATION,COORDINATES,TYPE OF SITE,ELEVATION,PERIOD,CULTURAL COMPLEX,ABSOLUTE DATING,STONE ARTIFACTS,PALEOLITHIC HUMAN REMAINS,LITERATURE,coord_ns,coord_ew
0,Abri Kontija 002,Istra,Limski kanal,"45° 8'15""N, 13°43'7"" E",RS,46,UP,G,/,YES,NO,"(Janković et al. 2016, 2017a; b, 2022; Peresan...",45.137500,13.718611
1,Abri Šebrn\r\n,Istria,Ćićarija,"45.33771253384577, 14.162687241350081",RS,750,MES,EMes,8650±9360 C-14 bp or 7610±8400 Cal BC,YES,NO,(Miracle et al. 2000; Komšo 2006: 64; Komšo et...,45.337713,14.162687
2,Brjgućeva Loza 1 (Loza),Istra,Kastav,"45° 28' 4"" N, 14° 14' 32"" E",C,510,MES,NaN,NaN,YES,NO,(Malez 1979: 242–43 karta 1:52; Paunović et al...,45.467778,14.242222
3,Borik,Dalmacija,Zadar (Oštri rat ili Punta Mika)\r\n,"44° 05'N, 15° 15' E",O,5,MES,NaN,NaN,YES,NO,(Malez 1979: 227 karta 1:76; Paunović et al. 2...,44.083333,15.250000
4,Brodski Drenovac,Slavonija,brodsko Posavlje (Pleternica),"45° 13'N, 17° 45'E",O,248,"UP, Mez",NaN,NaN,YES,NO,(Malez 1979: 227 karta 1:50; Komšo 2006: 76),45.216667,17.750000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,"Vuglovec, Ivanec",SZ Hrvatska,"Ravna gora, planina Ivančica","46° 12'N, 16° 08'E",C,?,EUP,"AU,G",NaN,NaN,NO,(Malez 1979: 274 karta 1:39; Paunović et al. 2...,46.200000,16.133333
64,Zamet,Kvarner,Rijeka,"45.34709703002665, 14.385885538632936",C,140m,UP,/,30-20ka- not absolute,YES,NO- soot traces,Malez?,45.347097,14.385886
65,Zapuntelsko polje,Dalmacija,otok Molat,"44° 15' N, 14° 46' E",O,/,UP,"M, Epipaleolithic",/,/,NO,(Malez 1979: 274–75 karta 1:70; Paunović et al...,44.250000,14.766667
66,"Zarilac, Požeška kotlina",Slavonija,Požeška kotlina,"45° 25' N, 17° 53' E",O,/,UP,"AU,G",NaN,YES,NO,(Malez 1979: 275–76 karta 1:46; Paunović et al...,45.416667,17.883333


In [5]:
fail = test[(test['coord_ns'] == 0) & (test['coord_ew'] == 0)]
correct = test[(test['coord_ns'] != 0) & (test['coord_ew'] != 0)]

fail

,NAME,GEOGRAPHICAL REGION,GEOGRAPHICAL LOCATION,COORDINATES,TYPE OF SITE,ELEVATION,PERIOD,CULTURAL COMPLEX,ABSOLUTE DATING,STONE ARTIFACTS,PALEOLITHIC HUMAN REMAINS,LITERATURE,coord_ns,coord_ew
61,Vorganjska peć,Kvarner,otok Krk,/,C,251,MP?,M?,NaN,YES,NaN,(Malez 1979: 274 karta 1:65; Sirovica et al. 2...,0.0,0.0


In [6]:
correct.describe()

,coord_ns,coord_ew
count,67.000000,67.000000
mean,44.732567,15.341171
std,0.870159,1.217096
min,42.970325,13.238056
25%,44.059911,14.376276
50%,44.833333,15.028095
75%,45.333333,16.282021
max,46.303333,18.016667


In [7]:
correct.to_excel("KnownSites_normalized.xlsx", index=False)